In [111]:
import pandas as pd
import numpy as np
import dataclasses

In [112]:
df = pd.read_excel('data/prelim_maps_data.xlsx')
df.head()

,Map Name,Matchup,Replay Version,Games,Wins,Losses,Team 1 %,Team 2 %
0,16-Bit LE,PvT,4.4.1.66668,3716,1930,1763,0.5226,0.4774
1,16-Bit LE,PvT,4.3.0.64469,434,216,215,0.5012,0.4988
2,16-Bit LE,PvT,4.4.0.65895,4508,2255,2223,0.5036,0.4964
3,16-Bit LE,PvT,4.5.0.67188,1194,609,575,0.5144,0.4856
4,16-Bit LE,PvT,4.3.1.65094,2724,1334,1384,0.4908,0.5092


In [113]:
df.dtypes

Map Name           object
Matchup            object
Replay Version     object
Games               int64
Wins                int64
Losses              int64
Team 1 %          float64
Team 2 %          float64
dtype: object

In [114]:
df.describe()

,Games,Wins,Losses,Team 1 %,Team 2 %
count,1559.000000,1559.000000,1559.000000,1559.000000,1559.000000
mean,3377.927518,1707.390635,1658.802437,0.504538,0.495462
std,4301.848935,2198.549579,2100.886429,0.095456,0.095456
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1124.500000,557.500000,543.500000,0.483600,0.473250
50%,2435.000000,1213.000000,1197.000000,0.503600,0.496400
75%,4241.000000,2127.500000,2074.000000,0.526750,0.516400
max,40612.000000,20609.000000,20204.000000,1.000000,1.000000


In [115]:
df['Replay Version'].iloc[4]

'4.3.1.65094'

# List of things to do

* Create a list of all the major patches

1. Clean up replay versions to match major patches
2. Group by replay versions
3. Remove win and loss percentages
4. Append pillar data
5. Begin data analysis

In [116]:
# List of major balance updates via Team Liquid
# https://liquipedia.net/starcraft2/Patches
patches_list = [
    '3.8',
    '3.11', 
    '3.12',
    '3.14',
    '4.0', 
    '4.1.1', 
    '4.1.4', 
    '4.2.1', 
    '4.3', 
    '4.8.3', 
    '4.10.1',
    '4.11.4',
    '5.0.2'
]

In [117]:
# Create function to match replay version to major patch version

def assign_patch(version):
    """
    This function takes the replay version and assigns the according balance patch from the patch list.
    Link: https://liquipedia.net/starcraft2/Patches
    """
    ver = version.split('.')
    
    if ver[0] == '3':
        if int(ver[1]) < 11:
            return '3.8'
        elif int(ver[1]) == 11:
            return '3.11'
        elif int(ver[1]) < 14:
            return '3.12'
        elif int(ver[1]) <= 19:
            return '3.14'
    elif ver[0] == '4':
        if ver[1] == '0':
            return '4.0'
        elif ver[1] == '1':
            if ver[2] == '1':
                return '4.1.1'
            elif int(ver[2]) < 4:
                return '4.1.1'
            elif int(ver[2][0]) <= 9:
                return '4.1.4'
        elif int(ver[1]) < 3:
            return '4.2.1'
        elif int(ver[1]) < 8:
            return '4.3'
        elif ver[1] == '8':
            if int(ver[2]) < 3:
                return '4.3'
            else:
                return '4.8.3'
        elif int(ver[1]) < 10:
            return '4.8.3'
        elif ver[1] == '10':
            if ver[2] == '0':
                return '4.8.3'
            else:
                return '4.10.1'
        elif ver[1] == '11':
            if int(ver[2]) < 4:
                return '4.10.1'
            else:
                return '4.11.4'
        elif int(ver[2]) < 13:
            return '4.11.4'
    elif ver[0] == '5':
        if int(ver[2]) < 2:
            return '4.11.4'
        elif int(ver[2]) >= 2:
            return '5.0.2'
    
    else:
        print('something broke')

In [118]:
df['patch'] = df['Replay Version'].apply(assign_patch)

In [119]:
df.head()

,Map Name,Matchup,Replay Version,Games,Wins,Losses,Team 1 %,Team 2 %,patch
0,16-Bit LE,PvT,4.4.1.66668,3716,1930,1763,0.5226,0.4774,4.3
1,16-Bit LE,PvT,4.3.0.64469,434,216,215,0.5012,0.4988,4.3
2,16-Bit LE,PvT,4.4.0.65895,4508,2255,2223,0.5036,0.4964,4.3
3,16-Bit LE,PvT,4.5.0.67188,1194,609,575,0.5144,0.4856,4.3
4,16-Bit LE,PvT,4.3.1.65094,2724,1334,1384,0.4908,0.5092,4.3


In [120]:
df = df[['Map Name', 'Matchup', 'Wins', 'Losses', 'patch']].copy()
df.head(n=10)

,Map Name,Matchup,Games,Wins,Losses,patch
0,16-Bit LE,PvT,3716,1930,1763,4.3
1,16-Bit LE,PvT,434,216,215,4.3
2,16-Bit LE,PvT,4508,2255,2223,4.3
3,16-Bit LE,PvT,1194,609,575,4.3
4,16-Bit LE,PvT,2724,1334,1384,4.3
5,16-Bit LE,PvT,2,1,1,4.3
6,16-Bit LE,PvT,4063,2026,2013,4.3
7,16-Bit LE,PvZ,3253,1660,1576,4.3
8,16-Bit LE,PvZ,3685,1863,1804,4.3
9,16-Bit LE,PvZ,3479,1798,1663,4.3


In [121]:
df = df.groupby(['Map Name', 'Matchup', 'patch']).sum().reset_index() # group values by patch
df.head(n=10)

,Map Name,Matchup,patch,Games,Wins,Losses
0,16-Bit LE,PvT,4.3,16641,8371,8174
1,16-Bit LE,PvZ,4.3,14687,7538,7069
2,16-Bit LE,TvZ,4.3,17548,9424,8012
3,Abiogenesis LE,PvT,4.1.4,5084,2523,2530
4,Abiogenesis LE,PvT,4.2.1,8568,4363,4140
5,Abiogenesis LE,PvT,4.3,2461,1289,1159
6,Abiogenesis LE,PvZ,4.1.4,5363,2339,2997
7,Abiogenesis LE,PvZ,4.2.1,9074,4147,4877
8,Abiogenesis LE,PvZ,4.3,2499,1105,1382
9,Abiogenesis LE,TvZ,4.1.4,6222,3097,3084


In [127]:
8371+8174

16545

In [122]:
df['Games'] = df.Wins + df.Losses
df['win_perc'] = round((df.Wins / df.Games), 4) # assign rounded win percentage
df['loss_perc'] = round((df.Losses / df.Games), 4) # assign rounded loss percentage

In [123]:
df.head()

,Map Name,Matchup,patch,Games,Wins,Losses,win_perc,loss_perc
0,16-Bit LE,PvT,4.3,16641,8371,8174,0.5030,0.4912
1,16-Bit LE,PvZ,4.3,14687,7538,7069,0.5132,0.4813
2,16-Bit LE,TvZ,4.3,17548,9424,8012,0.5370,0.4566
3,Abiogenesis LE,PvT,4.1.4,5084,2523,2530,0.4963,0.4976
4,Abiogenesis LE,PvT,4.2.1,8568,4363,4140,0.5092,0.4832


In [124]:
df = df.sort_values(['patch', 'Map Name']) # sort values by patch and group by map name

In [125]:
df.head()

,Map Name,Matchup,patch,Games,Wins,Losses,win_perc,loss_perc
12,Abyssal Reef LE,PvT,3.11,2236,1070,1162,0.4785,0.5197
19,Abyssal Reef LE,PvZ,3.11,2606,1260,1340,0.4835,0.5142
26,Abyssal Reef LE,TvZ,3.11,3587,1656,1916,0.4617,0.5342
87,Bel'Shir Vestige LE (Void),PvT,3.11,2526,1276,1243,0.5051,0.4921
90,Bel'Shir Vestige LE (Void),PvZ,3.11,2701,1285,1407,0.4757,0.5209


In [126]:
df.to_excel('data\maps_data.xlsx')